In [ ]:
import pandas as pd
import mapply
mapply.init(n_workers=30, progressbar=True, max_chunks_per_worker=100)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/opt/models/distilroberta-base/", do_lower_case = False)

In [ ]:
# lyrics = pd.read_csv('./../data/lyrics.csv')
# lyrics = lyrics[lyrics['tag'] != 'misc']

# import re

# del_sqbrackets = re.compile(r'\[.*?\]')
# lyrics['lyrics'] = lyrics['lyrics'].mapply(lambda x: del_sqbrackets.sub('', x))
# lyrics['tokenized'] = lyrics['lyrics'].mapply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=10000, truncation=True))
# lyrics.to_feather('tk_distil.feather')




---




In [ ]:
lyrics = pd.read_feather('tk_distil.feather')

ONE_CLASS = ''

if ONE_CLASS != '':
  positive = lyrics[lyrics['tag'] == ONE_CLASS]
  negative = lyrics[lyrics['tag'] != ONE_CLASS].sample(n=positive.shape[0])
  negative['tag'] = f'not_{ONE_CLASS}'
  lyrics = pd.concat([positive, negative])

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import torch


X_tensor_full = pad_sequence(
  [torch.tensor(x) for x in lyrics['tokenized']],
  batch_first=True,
  padding_value=tokenizer.pad_token_id
)
attention_full = X_tensor_full != tokenizer.pad_token_id
attention_full = attention_full.to(torch.float32)

X_tensor = X_tensor_full[:, :100]
attention = attention_full[:, :100]

y_tensor = torch.tensor(lyrics['tag'].astype('category').cat.codes.tolist())

In [ ]:
TRAIN_SIZE = 0.6
VAL_SIZE = 0.2
TEST_SIZE = 0.2

tot_len = len(X_tensor)
torch.random.manual_seed(42)
shuffle = torch.randperm(tot_len)

X_tensor = X_tensor[shuffle]
attention = attention[shuffle]
y_tensor = y_tensor[shuffle]

train_size = int(len(X_tensor) * TRAIN_SIZE)
val_size = int(len(X_tensor) * VAL_SIZE)
test_size = len(X_tensor) - train_size - val_size

X_train = X_tensor[:train_size]
X_val = X_tensor[train_size:train_size + val_size]
X_test = X_tensor[train_size + val_size:]

attention_train = attention[:train_size]
attention_val = attention[train_size:train_size + val_size]
attention_test = attention[train_size + val_size:]

y_train = y_tensor[:train_size]
y_val = y_tensor[train_size:train_size + val_size]
y_test = y_tensor[train_size + val_size:]

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 128

train_data = TensorDataset(X_train, attention_train, y_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

val_data = TensorDataset(X_val, attention_val, y_val)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=BATCH_SIZE)

In [ ]:
from transformers import AutoModelForSequenceClassification, AdamW

model = AutoModelForSequenceClassification.from_pretrained(
  "/opt/models/distilroberta-base/",
  num_labels=lyrics['tag'].nunique(),
  output_attentions=False,
  output_hidden_states=False,
)

model.cuda();

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-4)

from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 40

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
# for name, param in model.named_parameters():
#   if 'classifier' not in name:
#     param.requires_grad = False
#   else:
#     print(name, param.requires_grad)

In [ ]:
import random
import numpy as np
from tqdm import tqdm

device = torch.device("cuda")

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss = 0
    eval_f1 = torch.zeros(lyrics['tag'].nunique())
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in tqdm(val_dataloader, total=len(val_dataloader)):
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/160bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        pred = outputs.logits.detach().cpu().argmax(axis=1)
        true = b_labels.to('cpu')

        from torcheval.metrics.functional import multiclass_f1_score

        tmp_f1 = multiclass_f1_score(true, pred, num_classes=lyrics['tag'].nunique() , average='macro')
        
        # Accumulate the total accuracy.
        eval_f1 += tmp_f1

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    f1 = eval_f1/nb_eval_steps
    for i in range(lyrics['tag'].nunique()):
      print(f'  F1 for {lyrics["tag"].astype("category").cat.categories[i]}: {f1[i]:.2f}', end=' ')
    print(f'  Mean F1: {f1.mean():.2f}')
    # print("  F1: {0:.2f}".format(eval_f1/nb_eval_steps))
    # print("  F1: {0:.2f}".format(eval_f1/nb_eval_steps))

print("")
print("Training complete!")